In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd

In [2]:
def min_max_normalize(lst):
    """
        Helper function for movielens dataset, not useful for discrete multi class clasification.

        Return:
        Normalized list x, in range [0, 1]
    """
    maximum = max(lst)
    minimum = min(lst)
    toreturn = []
    for i in range(len(lst)):
        toreturn.append((lst[i]- minimum)/ (maximum - minimum))
    return toreturn

In [3]:
def z_standardize(X_inp):
    """
        Z-score Standardization.
        Standardize the feature matrix, and store the standarize rule.

        Parameter:
        X_inp: Input feature matrix.

        Return:
        Standardized feature matrix.
    """
    
    toreturn = X_inp.copy()
    for i in range(X_inp.shape[1]):
        std = np.std(X_inp[:, i])               # ------ Find the standard deviation of the feature
        mean = np.mean(X_inp[:, i])             # ------ Find the mean value of the feature
        temp = []
        for j in np.array(X_inp[:, i]):
            
            """    
                #TODO: 1. implement the standardize function
            """

            temp += [(j - mean) / std]
        toreturn[:, i] = temp
    return toreturn

In [4]:
def sigmoid(x):
    """ 
        Sigmoid Function

        Return:
        transformed x.
    """
    """    
        #TODO: 2. implement the sigmoid function
    """
    return 1 / (1 + np.exp(-x))

In [5]:
def logistic_loss(y, y_hat):
    loss = - (y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)) 
    return np.mean(loss)

In [8]:
class Logistic_Regression():
    
    def __init__(self):
        """
            Some initializations, if necessary
        """
        self.model_name = 'Logistic Regression'
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and do normalization to y_train
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                y_train: Matrix or 2-D array. Input target value.
        """
        self.X = X_train
        self.y = y_train
 
        count = 0
        uni = np.unique(y_train)
        for y in y_train:
            if y == min(uni):
                self.y[count] = 0
            else:
                self.y[count] = 1
            count += 1        
        
        n, m = X_train.shape
        self.theta = np.zeros(m)
        self.b = 0
    
    def gradient(self, X_inp, y_inp, theta, b):
        """
            negative log likelihood
            Calculate the gradient of Weight and Bias, given sigmoid_yhat, true label, and data

            Parameter:
                X_inp: Matrix or 2-D array. Input feature matrix.
                y_inp: Matrix or 2-D array. Input target value.
                theta: Matrix or 1-D array. Weight matrix.
                b: int. Bias.

            Return:
                grad_theta: gradient with respect to theta
                grad_b: gradient with respect to b

        NOTE: There are several ways of implementing the gradient. We are merely providing you one way
        of doing it. Feel free to change the code and implement the way you want.
        """
        z = X_inp @ theta + b
        hz = sigmoid(z)
        grad_theta = (1 / len(y_inp)) * np.dot(X_inp.T, (hz - y_inp))
        grad_b = (1 / len(y_inp)) * np.sum(hz - y_inp)
        return grad_theta, grad_b

    def gradient_descent_logistic(self, alpha, num_pass, early_stop=0, standardized=True):
        """
            Logistic Regression with gradient descent method

            Parameter:
                alpha: (Hyper Parameter) Learning rate.
                num_pass: Number of iteration
                early_stop: (Hyper Parameter) Least improvement error allowed before stop. 
                            If improvement is less than the given value, then terminate the function and store the coefficients.
                            default = 0.
                standardized: bool, determine if we standardize the feature matrix.
                
            Return:
                self.theta: theta after training
                self.b: b after training
        """
        if standardized:
            self.X = z_standardize(self.X)
        
        n, m = self.X.shape
        self.error = []
        temp_theta = self.theta.copy()
        temp_b = self.b
        for i in range(num_pass):    
            
            grad_theta, grad_b = self.gradient(self.X, self.y, temp_theta, temp_b)

            temp_theta = temp_theta - alpha * grad_theta
            temp_b = temp_b - alpha * grad_b

            temp_y_hat = sigmoid(self.X @ temp_theta + temp_b)
            temp_error = -np.mean(self.y * np.log(temp_y_hat) + (1 - self.y) * np.log(1 - temp_y_hat))
            if len(self.error) >= 1:
                pre_error = self.error[-1] 
                if (abs(pre_error - temp_error) < early_stop) or (abs(abs(pre_error - temp_error) / pre_error) < early_stop):
                    return temp_theta, temp_b
            self.error += [temp_error]
            self.theta = temp_theta
            self.b = temp_b
        
        return self.theta, self.b
    
    def predict_ind(self, x):
        """
            Predict the most likely class label of one test instance based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                p: prediction of given data point
        """
        p = sigmoid(np.dot(x, self.theta))  # Calculate probability
        return p
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                p: prediction of given data matrix
        """
        
        """
            TODO: 8. Revise the following for-loop to call predict_ind to get predictions.
        """
        X = np.mat(X)
        ret = []                  # -------- Use predict_ind to generate the prediction list
        for x in X:
            ret.append(self.predict_ind(x))
        return ret


In [9]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
#names = ['f_acid', 'v_acid', 'c_acid', 'sugar', 'chlorides', 'f_SO2', 't_SO2', 'density', 'ph', 'sulphates', 'alcohol', 'quality']
wine = pd.read_csv(url_Wine, delimiter=';')

In [10]:
wine5 = wine.loc[wine.quality == 5]
wine6 = wine.loc[wine.quality == 6]
wineall = pd.concat([wine5,wine6])
wineall

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,6.3,0.51,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1593,6.8,0.62,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1595,5.9,0.55,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.51,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6


In [13]:
X = np.array(wineall.iloc[:,:10])
Y = np.array(wineall.quality)

In [14]:
count = 0
for y in Y:
    if y == 5:
        Y[count] = -1
    else:
        Y[count] = 1
    count += 1

In [15]:
logit = Logistic_Regression()
logit.fit(X, Y)

In [16]:
g = logit.gradient_descent_logistic(0.01, 50000,0)

In [17]:
w, b = g

In [18]:
g

(array([ 0.88912383, -0.39974625, -0.15983202,  0.3372175 , -0.16077521,
         0.22246689, -0.61888426, -0.86664095,  0.43059502,  0.54169442]),
 -0.07736234147666998)

In [19]:
hat = np.array(w.dot(z_standardize(X).T) + b)

In [20]:
hat1 = sigmoid(hat)
hat1

array([0.31149998, 0.22190842, 0.26046995, ..., 0.73781599, 0.66317442,
       0.6277219 ])

In [21]:
count = 0
for i in range(len(hat)):
    if hat1[i] < 0.5:
        if Y[i] == 0:
            count += 1
    else:
        if Y[i] == 1:
            count += 1
count

928

In [22]:
928/1319

0.7035633055344959